In [33]:
import pandas as pd
import os
import re

In [51]:
def clean_data(raw_path):
    """
    Concatenate raw dataframes with <s> token indicating the start of the collection and save the series of executables into dataframe

    Parameters:
    raw_path (str): path of raw dataframes
    """
    datasets = []
    # directory = os.fsencode(raw_path)
    # start = pd.Series(['<s>'])
    for i in range(1, 72):
        file_path = f"{raw_path}/df{i}.csv"
        try:
            df = pd.read_csv(file_path, delimiter="\t")
        except:
            continue
        temp = df[df['ID_INPUT'] == 3][['MEASUREMENT_TIME', 'VALUE']].reset_index(drop=True)
        temp = temp.rename(columns={'MEASUREMENT_TIME': 'Start', 'VALUE': 'Value'})
        temp['Start'] = pd.to_datetime(temp['Start']).dt.tz_localize(tz='GMT+0').dt.tz_convert('America/Los_Angeles').dt.tz_localize(None)
        temp['End'] = temp['Start'].shift(-1)
        temp['Duration'] = (temp['Start'].shift(-1) - temp['Start'])
        temp = temp.drop(len(temp)-1)
        datasets.append(temp)
    output_dataset = pd.concat(datasets).sort_values(by='Start').reset_index(drop=True)
    # print(output_dataset['Duration'].dtype)
    output_dataset.to_csv('/Users/yikaimao/Desktop/DSC_180B/Intel-capstone/data/processed/new_lstm_dataset_local.csv', index=False)
    return output_dataset

In [52]:
lstm_dataset = clean_data('/Users/yikaimao/Desktop/DSC_180B/Intel-capstone/data/raw')
lstm_dataset

,Start,Value,End,Duration
0,2022-12-12 12:34:09.893,VsDebugConsole.exe,2022-12-12 12:34:12.896,0 days 00:00:03.003000
1,2022-12-12 12:34:12.896,firefox.exe,2022-12-12 12:52:55.024,0 days 00:18:42.128000
2,2022-12-12 12:52:55.024,VsDebugConsole.exe,2022-12-12 17:06:02.811,0 days 04:13:07.787000
3,2022-12-12 17:06:02.811,explorer.exe,2022-12-12 17:06:14.814,0 days 00:00:12.003000
4,2022-12-12 17:06:14.814,VsDebugConsole.exe,2022-12-12 17:06:56.818,0 days 00:00:42.004000
...,...,...,...,...
8483,2023-02-06 19:31:38.653,League of Legends.exe,2023-02-06 19:38:23.950,0 days 00:06:45.297000
8484,2023-02-06 19:38:23.950,firefox.exe,2023-02-06 19:38:25.419,0 days 00:00:01.469000
8485,2023-02-06 19:38:25.419,explorer.exe,2023-02-06 19:38:26.442,0 days 00:00:01.023000
8486,2023-02-06 19:38:26.442,League of Legends.exe,2023-02-06 19:49:39.717,0 days 00:11:13.275000


In [41]:
dirFiles = os.listdir(directory)
dirFiles.sort(key=lambda x: int(re.findall('\d+', x)[0]))

[b'df58.csv',
 b'df59.csv',
 b'df71.csv',
 b'df49.csv',
 b'df61.csv',
 b'df48.csv',
 b'df13.csv',
 b'df6.csv',
 b'df7.csv',
 b'df12.csv',
 b'df38.csv',
 b'df5.csv',
 b'df4.csv',
 b'df39.csv',
 b'df11.csv',
 b'df29.csv',
 b'df15.csv',
 b'df1.csv',
 b'Old',
 b'df14.csv',
 b'df28.csv',
 b'df16.csv',
 b'df3.csv',
 b'df2.csv',
 b'df17.csv',
 b'df26.csv',
 b'df32.csv',
 b'df33.csv',
 b'df27.csv',
 b'df31.csv',
 b'df19.csv',
 b'df18.csv',
 b'df24.csv',
 b'df34.csv',
 b'df9.csv',
 b'df8.csv',
 b'df21.csv',
 b'df35.csv',
 b'df37.csv',
 b'df36.csv',
 b'df22.csv',
 b'df45.csv',
 b'df51.csv',
 b'df44.csv',
 b'df52.csv',
 b'df46.csv',
 b'df47.csv',
 b'df53.csv',
 b'df57.csv',
 b'df43.csv',
 b'outputs',
 b'df42.csv',
 b'df56.csv',
 b'df54.csv',
 b'df55.csv',
 b'df41.csv']

In [93]:
lstm_dataset['Value'].unique()

array(['VsDebugConsole.exe', 'firefox.exe', 'explorer.exe', 'Teams.exe',
       'SearchApp.exe', 'LeagueClientUx.exe', 'Discord.exe',
       'notepad.exe', 'League of Legends.exe', 'ShellExperienceHost.exe',
       'vpnui.exe', 'vpndownloader.exe', 'notepad++.exe', 'cmd.exe',
       'devenv.exe', 'steam.exe', 'steamwebhelper.exe', 'Yakuza0.exe',
       'chrome.exe', 'Zoom.exe', 'Talkdesk.exe', 'ScreenClippingHost.exe',
       'msedge.exe', 'OUTLOOK.EXE', 'obs64.exe', 'vlc.exe', 'slack.exe',
       'WeChat.exe', 'RiotClientUx.exe', 'LeagueClient.exe',
       'LeagueCrashHandler.exe', 'audacity.exe', '7zFM.exe',
       'DB Browser for SQLite.exe', 'Shell_TrayWnd', 'Chrome_WidgetWin_1',
       'MSTaskListWClass', 'MozillaWindowClass', 'ToolbarWindow32',
       'Windows.UI.Core.CoreWindow', 'Button', '#32768', 'DirectUIHWND',
       '#32770', 'StartMenuExperienceHost.exe', 'WindowsCamera.exe',
       'ApplicationFrameHost.exe', 'iCloud.exe', 'iCloudServices.exe',
       'Taskmgr.exe', 'Mic

In [37]:
a = 'df12.csv'
re.findall('\d+', a)

['12']